In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping

In [ ]:
# DATA load
X = pd.read_csv('/loan_train_preprocessed.csv')

# backward
X = X[['term', 'initial_list_status', 'int_rate', 
'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 
'inq_last_6mths', 'revol_util', 'recoveries', 
'collection_recovery_fee', 'tot_cur_bal', 
'home_ownershipRENT', 'purposesmall_business', 
'purposewedding', 'earliest_cr_line2000']]

y = pd.read_csv('/loan_train_label.csv')
y = y.drop(['id'], axis=1)

# 새 섹션

In [ ]:
# Dividing the data into train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.20 )
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size = 0.20 )

x_train.shape,y_train.shape,x_val.shape,y_val.shape,x_test.shape,y_test.shape

((10240, 16), (10240, 1), (2560, 16), (2560, 1), (3200, 16), (3200, 1))

In [ ]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
import warnings
warnings.filterwarnings("ignore")

model = Sequential()
model.add(Dense(2048, input_shape = (len(x_train.columns),), activation='ELU'))
model.add(Dense(1024, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='ELU'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='ELU'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy', f1])

In [ ]:
# Adjust the weights of the classes since your dataset is HIGHLY IMBALANCED!
class_weight = {0: 1.,
                1: 1.}

model.fit(x_train.values,
         y_train.values,
         epochs = 300,
         batch_size = 2048,
         validation_data = (x_val.values, y_val.values), class_weight=class_weight,
         callbacks=[EarlyStopping(monitor='val_f1', mode='max', patience=100, restore_best_weights=True)])


Epoch 1/300
5/5 [==============================] - 5s 815ms/step - loss: 0.6831 - accuracy: 0.5871 - f1: 0.5855 - val_loss: 0.6106 - val_accuracy: 0.6598 - val_f1: 0.6345
Epoch 2/300
5/5 [==============================] - 4s 749ms/step - loss: 0.6221 - accuracy: 0.6328 - f1: 0.6118 - val_loss: 0.5961 - val_accuracy: 0.6672 - val_f1: 0.6391
Epoch 3/300
5/5 [==============================] - 4s 742ms/step - loss: 0.6030 - accuracy: 0.6560 - f1: 0.6302 - val_loss: 0.5723 - val_accuracy: 0.6699 - val_f1: 0.6514
Epoch 4/300
5/5 [==============================] - 4s 741ms/step - loss: 0.5902 - accuracy: 0.6479 - f1: 0.6396 - val_loss: 0.5714 - val_accuracy: 0.6727 - val_f1: 0.6425
Epoch 5/300
5/5 [==============================] - 4s 742ms/step - loss: 0.5794 - accuracy: 0.6595 - f1: 0.6375 - val_loss: 0.5650 - val_accuracy: 0.6707 - val_f1: 0.6753
Epoch 6/300
5/5 [==============================] - 4s 747ms/step - loss: 0.5731 - accuracy: 0.6620 - f1: 0.6382 - val_loss: 0.5616 - val_accuracy

In [ ]:
y_prediction = model.predict(x_test.values)
y_prediction= [1 if i>=0.5 else 0 for i in y_prediction]
print("The Test Accuracy of the model is: {} %".format(accuracy_score(y_test.values, y_prediction) * 100.)) 
print()

print(confusion_matrix(y_test.values, y_prediction))
print()

target_names = ['Fully Paid', 'Charged Off']
print(classification_report(y_test, y_prediction, target_names=target_names))

The Test Accuracy of the model is: 68.09375 %

[[1036  554]
 [ 467 1143]]

              precision    recall  f1-score   support

  Fully Paid       0.69      0.65      0.67      1590
 Charged Off       0.67      0.71      0.69      1610

    accuracy                           0.68      3200
   macro avg       0.68      0.68      0.68      3200
weighted avg       0.68      0.68      0.68      3200

